## DeepDream - 深度理解神经网路结构及应用

In [23]:
from __future__ import print_function

import numpy as np
import scipy.misc
import tensorflow as tf
from PIL import Image

## 创建图和会话

In [24]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

D:\Anaconda3\envs\Python36\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## Inception模型（开源已训练完成模型）的加载

In [25]:
model_fn = "Data/Deep-Dream/tensorflow_inception_graph.pb"
with tf.gfile.FastGFile(model_fn, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [26]:
# 定义输入图像的占位符
t_input = tf.placeholder(np.float32, name="input")

# 图像预处理 - 减均值
# 开源Inception模型在训练时做了减均值预处理，此处也需坚强同样的均值以保持一致
imagenet_mean = 117.0

# 图像预处理 - 增加维度
# 图像数据格式一般是（height 高度，width 宽度，channels 通道），为同时将多张图片输入网络而在前面增加一维
# 变为（batch，height，width，channels）
# batch代表将多张图像送入网络
# 0代表在Tensor下标为0位置进行插入，即头插入，-1则代表从后往前插入，即尾插
# 若为1则为在下标0与2中间进行插入，往后同理
t_preprocessed = tf.expand_dims(t_input - imagenet_mean, 0)

# 导入模型并将预处理的图像送入网络中
tf.import_graph_def(graph_def, {"input": t_preprocessed})

## 找到卷积层

In [27]:
layers = [op.name for op in graph.get_operations() if op.type == "Conv2D" and "import/" in op.name]

# 卷积层层数
print("Number of layers: ", len(layers))

Number of layers:  59


In [28]:
# 输出所有卷积层名称
print(layers)

['import/conv2d0_pre_relu/conv', 'import/conv2d1_pre_relu/conv', 'import/conv2d2_pre_relu/conv', 'import/mixed3a_1x1_pre_relu/conv', 'import/mixed3a_3x3_bottleneck_pre_relu/conv', 'import/mixed3a_3x3_pre_relu/conv', 'import/mixed3a_5x5_bottleneck_pre_relu/conv', 'import/mixed3a_5x5_pre_relu/conv', 'import/mixed3a_pool_reduce_pre_relu/conv', 'import/mixed3b_1x1_pre_relu/conv', 'import/mixed3b_3x3_bottleneck_pre_relu/conv', 'import/mixed3b_3x3_pre_relu/conv', 'import/mixed3b_5x5_bottleneck_pre_relu/conv', 'import/mixed3b_5x5_pre_relu/conv', 'import/mixed3b_pool_reduce_pre_relu/conv', 'import/mixed4a_1x1_pre_relu/conv', 'import/mixed4a_3x3_bottleneck_pre_relu/conv', 'import/mixed4a_3x3_pre_relu/conv', 'import/mixed4a_5x5_bottleneck_pre_relu/conv', 'import/mixed4a_5x5_pre_relu/conv', 'import/mixed4a_pool_reduce_pre_relu/conv', 'import/mixed4b_1x1_pre_relu/conv', 'import/mixed4b_3x3_bottleneck_pre_relu/conv', 'import/mixed4b_3x3_pre_relu/conv', 'import/mixed4b_5x5_bottleneck_pre_relu/conv',

In [29]:
# 输出指定卷积层的参数
name = "mixed4d_3x3_bottleneck_pre_relu"
print("shape of %s:%s" % (name, str(graph.get_tensor_by_name("import/" + name + ":0").get_shape())))

name2 = "mixed4e_5x5_bottleneck_pre_relu"
print("shape of %s:%s" % (name2, str(graph.get_tensor_by_name("import/" + name2 + ":0").get_shape())))

shape of mixed4d_3x3_bottleneck_pre_relu:(?, ?, ?, 144)
shape of mixed4e_5x5_bottleneck_pre_relu:(?, ?, ?, 32)


## 生成原始Deep Dream图像（单通道）

In [30]:
# 把一个numpy.ndarray保存成图像文件
def savearray(img_array, img_name):
    # 如果想保存成RGB图片，则将上面的 'L'  改为 'RGB' 即可
    # Image.fromarray(img_array).convert("L").save(img_name)
    # Image.fromarray(img_array).save(img_name)
    scipy.misc.toimage(img_array).save(img_name)
    print("img saved: %s" % img_name)

In [31]:
# 渲染函数
# t_obj：是layer_output[:, :, :, channel]，即卷积层某个通道的值
# img0：初始图像（噪声图像）
# iter_n：迭代次数
# step：用于控制每次迭代步长，可以看作学习率
def render_naive_function(t_obj, img0, iter_n=20, step=1.0):
    # t_score是t_obj的平均值
    # 由于我们的目标是调整输出图像使卷积层激活值尽可能大
    # 即最大化t_score
    # 为达到此目标，可使用梯度下降
    # 计算t_score对t_input的梯度
    t_score = tf.reduce_mean(t_obj)

    t_grad = tf.gradients(t_score, t_input)[0]

    # 复制新图像可避免影响原图像的值
    img = img0.copy()

    for i in range(iter_n):
        # 在session中计算梯度，以及当前的t_score
        g, score = session.run([t_grad, t_score], feed_dict={t_input: img})

        # 对img应用梯度
        # 首先对梯度进行归一化处理
        # 1e-8：0.00000001，即1 * 10^(-8)
        g /= g.std() + 1e-8

        # 将正规化处理后的梯度应用在图像上，step用于控制每次迭代步长，此处为1.0
        img += g * step

        print("iter: %d" % (i + 1), "score(mean) = %f" % score)

    # 保存图片
    savearray(img, "Image/naive_deep_dream.jpg")

In [32]:
channel = 139
# "mixed4a_3x3_bottleneck_pre_relu"共144个通道
# 选取任意通道（0 ~ 143之间任意整数）进行最大化
layer_output = graph.get_tensor_by_name("import/%s: 0" % name)

# 定义噪声图像
image_noise = np.random.uniform(size=(224, 224, 3)) + 100.0

# 调用render_naive_function函数渲染
render_naive_function(layer_output[:, :, :, channel], image_noise, iter_n=20)

# 保存并显示图片
im = Image.open("Image/naive_deep_dream.jpg")
im.show()
im.save("Image/naive_single_chn.jpg")

iter: 1 score(mean) = -20.084309
iter: 2 score(mean) = -32.286839
iter: 3 score(mean) = 25.363852
iter: 4 score(mean) = 91.472725
iter: 5 score(mean) = 150.023575
iter: 6 score(mean) = 201.139053
iter: 7 score(mean) = 262.625397
iter: 8 score(mean) = 329.964844
iter: 9 score(mean) = 368.014954
iter: 10 score(mean) = 416.939453
iter: 11 score(mean) = 456.069672
iter: 12 score(mean) = 496.902832
iter: 13 score(mean) = 537.332153
iter: 14 score(mean) = 562.979187
iter: 15 score(mean) = 598.710266
iter: 16 score(mean) = 630.103699
iter: 17 score(mean) = 656.299927
iter: 18 score(mean) = 688.252625
iter: 19 score(mean) = 711.564026
iter: 20 score(mean) = 733.312195
img saved: Image/naive_deep_dream.jpg


D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


## 较低层单通道卷积特征生成Deep Dream图像

In [33]:
# 定义卷积层、通道数，并取出对应Tensor
name3 = "mixed3a_3x3_bottleneck_pre_relu"
layer_output = graph.get_tensor_by_name("import/%s:0" % name3)
print("shape of %s: %s" % (name3, str(graph.get_tensor_by_name("import/" + name3 + ":0").get_shape()),))

shape of mixed3a_3x3_bottleneck_pre_relu: (?, ?, ?, 96)


## 高层单通道卷积特征生成Deep Dream图像

In [34]:
# 定义卷积层、通道数，并取出对应Tensor
name4 = "mixed5b_5x5_pre_relu"
layer_output = graph.get_tensor_by_name("import/%s:0" % name4)
print("shape of %s: %s" % (name4, str(graph.get_tensor_by_name("import/" + name4 + ":0").get_shape()),))

shape of mixed5b_5x5_pre_relu: (?, ?, ?, 128)


In [35]:
# 定义噪声图像
img_noise = np.random.uniform(size=(224, 224, 3)) + 100.0

# 调用render_naive_function函数渲染
channel = 118
render_naive_function(layer_output[:, :, :, channel], img_noise, iter_n=20)

# 保存并显示照片
im = Image.open("Image/naive_deep_dream.jpg")
im.show()
im.save("Image/deep_single_chn.jpg")

iter: 1 score(mean) = -7.564476
iter: 2 score(mean) = -9.213534
iter: 3 score(mean) = -3.566386
iter: 4 score(mean) = 6.309409
iter: 5 score(mean) = 14.429076
iter: 6 score(mean) = 24.695652
iter: 7 score(mean) = 35.990284
iter: 8 score(mean) = 39.517696
iter: 9 score(mean) = 51.566605
iter: 10 score(mean) = 57.235104
iter: 11 score(mean) = 68.215401
iter: 12 score(mean) = 69.156860
iter: 13 score(mean) = 76.962822
iter: 14 score(mean) = 90.145866
iter: 15 score(mean) = 90.103256
iter: 16 score(mean) = 100.993958
iter: 17 score(mean) = 104.738289
iter: 18 score(mean) = 115.121605
iter: 19 score(mean) = 123.789383
iter: 20 score(mean) = 135.100174


D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


img saved: Image/naive_deep_dream.jpg


## 生成原始Deep Dream图像（所有通道）

In [36]:
# 定义卷积层、通道数，并取出对应Tensor
name = "mixed4d_3x3_bottleneck_pre_relu"
layer_output = graph.get_tensor_by_name("import/%s:0" % name)
print("shape of %s: %s" % (name, str(graph.get_tensor_by_name("import/" + name + ":0").get_shape()),))

# 定义噪声图像
img_noise = np.random.uniform(size=(224, 224, 3)) + 100.0

# 调用render_naive_function函数渲染
# 不指定特定通道，即表示利用所有通道特征
render_naive_function(layer_output, img_noise, iter_n=20)
# 单通道时：layer_output[:, :, :, channel]

# 保存并显示照片
im = Image.open("Image/naive_deep_dream.jpg")
im.show()
im.save("Image/all_chn.jpg")

shape of mixed4d_3x3_bottleneck_pre_relu: (?, ?, ?, 144)
iter: 1 score(mean) = -6.793727
iter: 2 score(mean) = -8.700486
iter: 3 score(mean) = -6.525362
iter: 4 score(mean) = 0.586559
iter: 5 score(mean) = 7.096675
iter: 6 score(mean) = 11.247742
iter: 7 score(mean) = 15.640568
iter: 8 score(mean) = 19.768738
iter: 9 score(mean) = 22.326097
iter: 10 score(mean) = 25.582514
iter: 11 score(mean) = 29.860027
iter: 12 score(mean) = 32.495106
iter: 13 score(mean) = 35.702557
iter: 14 score(mean) = 39.227608
iter: 15 score(mean) = 40.770065
iter: 16 score(mean) = 45.148392
iter: 17 score(mean) = 46.574661
iter: 18 score(mean) = 49.879055
iter: 19 score(mean) = 50.959644
iter: 20 score(mean) = 54.952278
img saved: Image/naive_deep_dream.jpg


D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


## 以背景图像为起点生成Deep Dream图像

In [37]:
# 定义卷积层、通道数，并取出对应Tensor
name = "mixed4c"
layer_output = graph.get_tensor_by_name("import/%s:0" % name)
print(layer_output)

Tensor("import/mixed4c:0", shape=(?, ?, ?, 512), dtype=float32, device=/device:CPU:0)


In [38]:
# 用一张背景图像（而不是随机噪音图像）作为起点对图像进化优化
img_test = Image.open("Image/IMG_0722.jpeg")

render_naive_function(layer_output, img_noise, iter_n=100)

# 保存并显示照片
im = Image.open("Image/naive_deep_dream.jpg")
im.show()
im.save("Image/new.jpg")

iter: 1 score(mean) = 5.419296
iter: 2 score(mean) = 10.414704
iter: 3 score(mean) = 19.698999
iter: 4 score(mean) = 29.727409
iter: 5 score(mean) = 38.942368
iter: 6 score(mean) = 46.909119
iter: 7 score(mean) = 54.018745
iter: 8 score(mean) = 60.285233
iter: 9 score(mean) = 65.990456
iter: 10 score(mean) = 71.035934
iter: 11 score(mean) = 75.265663
iter: 12 score(mean) = 79.099968
iter: 13 score(mean) = 82.899994
iter: 14 score(mean) = 85.147491
iter: 15 score(mean) = 88.846619
iter: 16 score(mean) = 90.967453
iter: 17 score(mean) = 93.590637
iter: 18 score(mean) = 95.463150
iter: 19 score(mean) = 97.497620
iter: 20 score(mean) = 99.496071
iter: 21 score(mean) = 101.037498
iter: 22 score(mean) = 102.938408
iter: 23 score(mean) = 104.367973
iter: 24 score(mean) = 105.550201
iter: 25 score(mean) = 107.192665
iter: 26 score(mean) = 108.457817
iter: 27 score(mean) = 109.990860
iter: 28 score(mean) = 111.066895
iter: 29 score(mean) = 111.910309
iter: 30 score(mean) = 113.125336
iter: 31 s

D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


## 定义相关函数

In [39]:
# 调整图像尺寸
def resize(img, hw):
    min = img.min()
    max = img.max()
    img = (img - min) / (max - min) * 255
    img = np.float32(scipy.misc.imresize(img, hw))
    img = img / 255 * (max - min) + min

    return img

In [40]:
# 将图像放大ratio倍
def resize_ratio(img, ratio):
    min = img.min()
    max = img.max()
    img = (img - min) / (max - min) * 255
    img = np.float32(scipy.misc.imresize(img, ratio))
    img = img / 255 * (max - min) + min

    return img

In [41]:
# 原始图像尺寸可能很大，从而导致内存耗尽问题
# 每次只对tile_size * tile_size 大小的图像计算梯度，避免内存问题
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)

    # 先在行上做整体移动，再在列上做整体移动
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
    grad = np.zeros_like(img)
    for y in range(0, max(h - sz // 2, sz), sz):
        for x in range(0, max(w - sz // 2, sz), sz):
            sub = img_shift[y:y + sz, x:x + sz]
            g = session.run(t_grad, {t_input: sub})
            grad[y:y + sz, x: x + sz] = g

    return np.roll(np.roll(grad, -sx, 1), -sy, 0)

In [42]:
# 优化图像后的渲染函数
def render_deep_dream_pro(t_obj, img0, iter_n=10, step=1.5, octave_n=4, octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]
    img = img0.copy()

    # 将图像进行金字塔分解
    # 从而分为高频，低频部分
    octaves = []
    for i in range(octave_n - 1):
        hw = img.shape[:2]
        lo = resize(img, np.int32(np.float32(hw) / octave_scale))
        hi = img - resize(lo, hw)
        img = lo
        octaves.append(hi)

    # 首先生成低频的图像，再依次放大并加上高频
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g * (step / (np.abs(g).mean() + 1e-7))

    img = img.clip(0, 255)
    savearray(img, "Image/new_pro.jpg")
    im = Image.open("Image/new_pro.jpg").show()

In [43]:
# 定义卷积层、通道数，并取出对应Tensor
name = "mixed4c"
layer_output = graph.get_tensor_by_name("import/%s:0" % name)
print("shape of %s: %s" % (name, str(graph.get_tensor_by_name("import/" + name + ":0").get_shape()),))

# 定义噪声图像
img0 = Image.open("Image/IMG_0722.jpeg")
img0 = np.float32(img0)

render_deep_dream_pro(tf.square(layer_output), img0)

shape of mixed4c: (?, ?, ?, 512)


D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  
D:\Anaconda3\envs\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


img saved: Image/new_pro.jpg
